# 為替レートと株価の相関 (2)

In [1]:
import datetime as dt
import japanize_matplotlib
import matplotlib.pyplot as plt
from matplotlib import dates as mdates
import mplfinance as mpf
import os
import pandas as pd
import yfinance as yf
import wget

## 共通用途の関数

In [2]:
# インデックスの時刻情報を取り除く
def conv_index_datetime_date(df: pd.DataFrame) -> pd.DataFrame:
    df.index = pd.to_datetime(['%d-%02d-%02d' % (ts.year, ts.month, ts.day) for ts in df.index])
    return df

In [3]:
# 東証上場銘柄情報の取得
def get_tickers_tse() -> pd.DataFrame:
    tse = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    filename = wget.download(tse)
    df = pd.read_excel(filename)[['コード', '銘柄名', '市場・商品区分']]
    os.remove(filename)
    return df

## データスコープと為替レートのインスタンス

In [4]:
start = '2024-05-01'
end = '2024-08-10'
ticker_exchange = yf.Ticker('USDJPY=X')

## １時間足の為替データを取得

In [5]:
df_hour = ticker_exchange.history(start=start, end=end, interval='1h')[['Open', 'High', 'Low', 'Close']]

# タイムゾーンを日本時間に変換
df_hour.index = df_hour.index.tz_convert('Asia/Tokyo')
df_hour

,Open,High,Low,Close
Datetime,,,,
2024-05-01 08:00:00+09:00,157.695999,157.751007,157.673004,157.746994
2024-05-01 09:00:00+09:00,157.737000,157.856003,157.677994,157.744003
2024-05-01 10:00:00+09:00,157.744003,157.848007,157.725998,157.800995
2024-05-01 11:00:00+09:00,157.798996,157.893005,157.787994,157.876999
2024-05-01 12:00:00+09:00,157.882996,157.934998,157.835007,157.912994
...,...,...,...,...
2024-08-09 06:00:00+09:00,147.222000,147.279999,147.154999,147.181000
2024-08-09 07:00:00+09:00,147.184998,147.307999,147.153000,147.289001
2024-08-09 08:00:00+09:00,147.272995,147.813004,147.235001,147.694000


### 終値を使うため、午後 14:00 のデータのみ抽出

In [6]:
list_day = sorted(list(set(['%d-%02d-%02d 14:00:00+09:00' % (ts.year, ts.month, ts.day) for ts in df_hour.index])))
list_series = list()
for ts in list_day:
    try:
        series = df_hour.loc[ts]
        list_series.append(series)
    except KeyError:
        print('no key of \'%s\'' % ts)
df_exchange = pd.concat(list_series, axis=1).T
print(df_exchange.head())

no key of '2024-05-04 14:00:00+09:00'
no key of '2024-05-11 14:00:00+09:00'
no key of '2024-05-18 14:00:00+09:00'
no key of '2024-05-25 14:00:00+09:00'
no key of '2024-06-01 14:00:00+09:00'
no key of '2024-06-08 14:00:00+09:00'
no key of '2024-06-15 14:00:00+09:00'
no key of '2024-06-22 14:00:00+09:00'
no key of '2024-06-29 14:00:00+09:00'
no key of '2024-07-06 14:00:00+09:00'
no key of '2024-07-13 14:00:00+09:00'
no key of '2024-07-20 14:00:00+09:00'
no key of '2024-07-27 14:00:00+09:00'
no key of '2024-08-03 14:00:00+09:00'
no key of '2024-08-09 14:00:00+09:00'
                                 Open        High         Low       Close
2024-05-01 14:00:00+09:00  157.854996  157.923004  157.817993  157.919998
2024-05-02 14:00:00+09:00  155.824005  155.889999  155.589996  155.695007
2024-05-03 14:00:00+09:00  152.912994  153.063004  152.843002  152.988007
2024-05-06 14:00:00+09:00  153.929993  153.977997  153.867996  153.921005
2024-05-07 14:00:00+09:00  154.472000  154.552994  154.37500

### インデックスの時刻情報を取り除く

In [7]:
df_exchange = conv_index_datetime_date(df_exchange)
df_exchange

,Open,High,Low,Close
2024-05-01,157.854996,157.923004,157.817993,157.919998
2024-05-02,155.824005,155.889999,155.589996,155.695007
2024-05-03,152.912994,153.063004,152.843002,152.988007
2024-05-06,153.929993,153.977997,153.867996,153.921005
2024-05-07,154.472000,154.552994,154.375000,154.539993
...,...,...,...,...
2024-08-02,149.259003,149.373001,148.886002,148.944000
2024-08-05,143.171997,143.487000,142.408997,142.660004
2024-08-06,145.507996,146.141998,145.100998,145.867004
2024-08-07,147.612000,147.813995,146.834000,146.979004


## 東証上場銘柄情報の取得

In [8]:
df_tse = get_tickers_tse()
df_tse

100% [........................................................] 810496 / 810496

,コード,銘柄名,市場・商品区分
0,1301,極洋,プライム（内国株式）
1,1305,ｉＦｒｅｅＥＴＦ ＴＯＰＩＸ（年１回決算型）,ETF・ETN
2,1306,ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,ETF・ETN
3,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN
4,1309,ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,ETF・ETN
...,...,...,...
4378,9991,ジェコス,プライム（内国株式）
4379,9993,ヤマザワ,スタンダード（内国株式）
4380,9994,やまや,スタンダード（内国株式）
4381,9996,サトー商会,スタンダード（内国株式）
